---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [249]:
# Try Random Forest first
# Should get rid of all input variables for the train set that aren't in the test set?
# Make compliance y_train, get rid of null entries

# NOTE: If not doing this on Coursera, would use smartystreets api (https://pypi.org/project/smartystreets.py/)
# to lookup mailing addresses and return lat/lon for all locations 
# Addresses .csv is only for the violation location, but I expect that payment rates would be lower for people
# that don't live in Detroit

"""
TODO:
    - Create df that drops the columns that aren't relevant/cause data leakage
    - Turn violation zip_code and resident address into lat/lon coords
    - Might have to transform violation_code/description into different integers with mapping
    - Have to sort disposition column, get rid of all that aren't responsible and have no fine or pending judgement
"""

# To suppress output
_ = ''

## Reasons for dropping columns

Drop the below because they are present in the train set but not the test set and would be a source of data leakage

['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail']

['ticket_id', 'violation_street_number', 'violation_street_name', 'violation_zip_code', 'mailing_address_str_number',
'mailing_address_str_name', 'zip_code', 'non_us_str_code', 'ticket_issued_date', 'hearing_date', 'state_fee',
'admin_fee', 'judgment_amount' 'grafitti_status', 'clean_up_cost', 'violation_description', 'address', 
'inspector_name', 'city']

**ticket_id**: irrelevant

**violation_street_number, violation_street_name, violation_zip_code**: have been replaced by lat and lon columns

**mailing_address_str_number, mailing_address_str_name, zip_code, non_us_str_code**
                                    : would be relevant to generate lat/lon if allowed by
                                        Coursera, would use api to achieve this
                                        
**violation_description**: linked to violation_code

**ticket_issued_date, hearing_date**: don't think these will be useful

**state_fee, admin_fee**: same in all cases

**judgment_amount**: irrelevant because of other columns

**grafitti_status**: potentially relevant, but only one instance in entire train df, therefore not enough info to help

**clean_up_cost**: same as above

**address**: covered with lat/lon

**inspector_name**: likely not relevant

**city**: state probably more relevant

In [ ]:
def blight_model():
    
    import pandas as pd
    import numpy as np
    from sklearn.ensemble import RandomForestClassifier
    
    # Create dataframes

    train_data = pd.read_csv('train.csv', encoding="ISO-8859-1", low_memory=False)
    test_data = pd.read_csv('test.csv')
    addresses = pd.read_csv('addresses.csv')
    geo = pd.read_csv('latlons.csv')
    
    # Fix nan values in geo data, hand-corrected

    geo['lat'].iloc[4126], geo['lon'].iloc[4126] = (42.376812505677535, -83.14313634711358)
    geo['lat'].iloc[10466], geo['lon'].iloc[10466] = (42.44676398408103, -83.02314581551171)
    geo['lat'].iloc[17293], geo['lon'].iloc[17293] = (42.36007354110193, -83.09577054390807)
    geo['lat'].iloc[34006], geo['lon'].iloc[34006] = (42.359132915414285, -83.15132718808694)
    geo['lat'].iloc[55750], geo['lon'].iloc[55750] = (42.35898711027719, -83.08120997318588)
    geo['lat'].iloc[74721], geo['lon'].iloc[74721] = (42.38342800142968, -83.05818415784229)
    geo['lat'].iloc[100359], geo['lon'].iloc[100359] = (42.340193798011214, -83.05855020202218)
    
    # Add lat/lon violation (not billing) coords data to train_data

    lat_lon = addresses.merge(geo, how='inner', on='address')
    train_data = train_data.merge(lat_lon, how='inner', on='ticket_id')
    
    # Get rid of columns in train_data that are not useful to the model, explained above

    columns_to_drop = ['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 
                       'compliance_detail', 'ticket_id', 'violation_street_number', 'violation_street_name', 
                       'violation_zip_code', 'mailing_address_str_number','mailing_address_str_name', 'zip_code', 
                       'non_us_str_code', 'ticket_issued_date', 'hearing_date', 'state_fee', 'admin_fee', 
                       'judgment_amount', 'grafitti_status', 'clean_up_cost', 'violation_description', 'address', 
                       'inspector_name', 'city']

    train_data = train_data.drop(columns_to_drop, axis=1)
    
    
    # Get rid of violators that were found not responsible (NaN in compliance field)

    rows_to_keep = train_data['compliance'].dropna().index
    train_data = train_data.iloc[rows_to_keep, :]
    
    # Create functions to turn string fields into integers to allow model to understand
    
    def df_str_to_int(df, col_name):
        """Take a dataframe column and turn each unique entry in the column into a unique number"""
        transform_dict = {}
        for i, item in enumerate(df[col_name].unique()):
            transform_dict[item] = i
        # Change the values of the dataframe to the unique numbers we want
        df[col_name] = df[col_name].apply(lambda x: key_to_value(x, transform_dict))


    def key_to_value(x, dic):
        """Obtain value from dict from the key"""
        return dic[x]
    

    # Iterate over the columns in the dataframe, changing only those that aren't numeric
    
    for col in train_data:
        try:
            int(train_data[col].iloc[0])
        except:
            df_str_to_int(train_data, col)

    # Drop any NaN if present and create X_train and y_train as np arrays
    
    train_data = train_data.dropna()
    y_train = np.array(train_data['compliance'])
    train_data = train_data.drop('compliance', axis=1)
    X_train = np.array(train_data)
    
    # Add lat/lon data to test_data df
    
    test_data = test_data.merge(lat_lon, how='inner', on='ticket_id')
    
    # Get rid of columns in the test_data df, same as before
    
    cols_to_drop = ['ticket_id', 'violation_street_number', 'violation_street_name', 
                'violation_zip_code', 'mailing_address_str_number','mailing_address_str_name', 
                'zip_code', 'non_us_str_code', 'ticket_issued_date', 'hearing_date', 'state_fee', 
                'admin_fee', 'judgment_amount', 'grafitti_status', 'clean_up_cost', 
                'violation_description', 'address', 'inspector_name', 'city']
    
    # Keep ticket_id for use as answer index later
    
    ticket_id = test_data['ticket_id']
    test_data = test_data.drop(cols_to_drop, axis=1)
    
    # Transform test_data as was done with train_data
    
    for col in test_data:
        try:
            int(test_data[col].iloc[0])
        except:
            df_str_to_int(test_data, col)

    # Drop any NaN that might be present
    
    X_test = np.array(test_data.dropna())
    
    # Train Random Forest Classifier
    
    clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
    
    # Create probability predictions from X_test for class compliance = True (1) and store in pd.Series
    
    res = clf.predict_proba(X_test)[:, 1]
    predictions = pd.Series(res, index=ticket_id, name='compliance')
    
    return predictions

In [ ]:
blight_model()